In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 46kB/s 
     |████████████████████████████████| 3.2MB 59.3MB/s 
     |████████████████████████████████| 491kB 47.5MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
!pip install tqdm

In [0]:
!pip install scikit-learn==0.20.3

     |████████████████████████████████| 5.4MB 24kB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [0]:
!pip install scikit-image==0.14.2

     |████████████████████████████████| 25.3MB 94kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.15.0
    Uninstalling scikit-image-0.15.0:
      Successfully uninstalled scikit-image-0.15.0


In [0]:
!pip install mtcnn==0.0.9

     |████████████████████████████████| 2.3MB 23kB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=88cff458e49ec625e0c684187fc3a36be64aff2da5619d9da7c45b8984fce9d9
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [0]:
!pip install numpy==1.16.3

     |████████████████████████████████| 17.3MB 141kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.4
    Uninstalling numpy-1.17.4:
      Successfully uninstalled numpy-1.17.4


In [0]:
cd 'drive/My Drive/upwork/Face_recognition-master'

/content/drive/My Drive/upwork/Face_recognition-master


In [0]:
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.svm import SVC, LinearSVC
#from sklearn.linear_model import SGDClassifier
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
#from sklearn.experimental import enable_hist_gradient_boosting
#from sklearn.ensemble import HistGradientBoostingClassifier
from joblib import dump, load
#from sklearn.naive_bayes import GaussianNB 
from VisionUtils import *
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
from mtcnn.mtcnn import MTCNN
from pathlib import Path
from sklearn.utils import shuffle
from tqdm import tqdm
import tensorflow as tf
#import autosklearn.classification

In [0]:
import sklearn
sklearn.__version__

'0.20.3'

In [0]:
np.__version__

'1.16.3'

In [0]:
print("TensorFlow Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Is GPU available: ", tf.test.is_gpu_available())

TensorFlow Version:  1.14.0
Eager mode:  False
Is GPU available:  False


##### !curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

In [0]:
def trans(feat1, feat2):
    f1 = [feat1[i1].reshape(-1,1) for i1 in range(len(feat1))]
    f2 = [feat2[i2].reshape(-1,1) for i2 in range(len(feat2))]
    cos_d = np.array([feat_distance_cosine_scalar(f1[i].T, f2[i]) for i in range(len(feat1))])
    cos_d = cos_d.reshape(-1,1)
    sqr_diff = np.power(np.abs(feat1- feat2), 2)
    rat = feat1/feat2
    data = np.hstack([cos_d, sqr_diff, rat])
    return data

train_aug_feat1 = np.load("features/train_aug_feat12.npy")
train_aug_feat2 = np.load("features/train_aug_feat22.npy")
train_aug_labels = np.load("features/train_aug_labels2.npy")

train_parallel_feat1 = np.load("features/train_parallel_feat11.npy")
train_parallel_feat2 = np.load("features/train_parallel_feat21.npy")
train_parallel_labels = np.load("features/train_parallel_labels.npy")

train_spotlight_feat1 = np.load("features/train_spotlight_feat11.npy")
train_spotlight_feat2 = np.load("features/train_spotlight_feat21.npy")
train_spotlight_labels = np.load("features/train_spotlight_labels.npy")

In [0]:
train_feat1_mbyt = np.load("data/processed_2/packaged/features/train_feat1.npy")
train_feat2_mbyt = np.load("data/processed_2/packaged/features/train_feat2.npy")
train_labels_mbyt = np.load("data/processed_2/packaged/features/train_labels.npy")

test_feat1_mbyt = np.load("data/processed_2/packaged/features/test_feat1.npy")
test_feat2_mbyt = np.load("data/processed_2/packaged/features/test_feat2.npy")
test_labels_mbyt = np.load("data/processed_2/packaged/features/test_labels.npy")

train_feat1_aug_mbyt = np.load("data/processed_2/packaged/motion_blur_features/train_feat1.npy")
train_feat2_aug_mbyt = np.load("data/processed_2/packaged/motion_blur_features/train_feat2.npy")
train_labels_aug_mbyt = np.load("data/processed_2/packaged/motion_blur_features/train_labels.npy")

test_feat1_aug_mbyt = np.load("data/processed_2/packaged/motion_blur_features/test_feat1.npy")
test_feat2_aug_mbyt = np.load("data/processed_2/packaged/motion_blur_features/test_feat2.npy")
test_labels_aug_mbyt = np.load("data/processed_2/packaged/motion_blur_features/test_labels.npy")




In [0]:
pwd

'/content/drive/My Drive/upwork/Face_recognition-master'

In [0]:
labels = \
  list(train_labels_mbyt) + \
  list(train_labels_aug_mbyt)  + \
  list(test_labels_mbyt)


In [0]:
len(labels)

26640

In [0]:
len(labels)

654177

In [0]:
train_feat1 = np.vstack([
                         train_feat1_mbyt,
                         train_feat1_aug_mbyt
])
train_feat2 = np.vstack([
                         train_feat2_mbyt,
                         train_feat2_aug_mbyt
])


In [0]:
len(train_labels_mbyt), len(train_feat1_mbyt)

(11840, 11840)

In [0]:
xtrain = trans(train_feat1, train_feat2)
xtest = trans(test_feat1_mbyt, test_feat2_mbyt)

In [0]:
tr_len = len(xtrain)
tst_len = len(xtest)

In [0]:
tr_len

23680

In [0]:

tst_len

2960

In [0]:
data = np.vstack([xtrain, xtest])


In [0]:
len(data)

26640

In [0]:
data.shape
#labels.shape

(26640, 257)

In [0]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
f_selector = SelectKBest(chi2, k = int(0.8*(len(data.T))))
data_ = f_selector.fit_transform(scaled_data, labels )

data_df = pd.DataFrame(data_)
image_paths_csv = pd.read_csv("features/image_paths.csv", index_col = False)

all_data_df = pd.concat([image_paths_csv,data_df], axis = 1)

len(all_data_df)

xtrain, xtest, ytrain, ytest = train_test_split(all_data_df, labels, shuffle = True, random_state = 42)

paths_dict = {
    "train_paths1": xtrain["path1"],
    "train_paths2": xtrain["path2"],
    "test_paths1": xtest["path1"],
    "test_paths2": xtest["path2"]
}
xtrain1 = xtrain.drop(["path1", "path2", "Unnamed: 0"], axis = 1).values
xtest1 = xtest.drop(["path1", "path2", "Unnamed: 0"], axis = 1).values

In [0]:
xtrain, xtest = data_[:tr_len], data_[-tst_len:]
ytrain, ytest = labels[:tr_len], labels[-tst_len:]
xtrain, ytrain = shuffle(xtrain, ytrain)

In [0]:
xtrain.shape

(23680, 205)

In [0]:
xtest.shape

(2960, 205)

Gradient Boosting Classifier

In [0]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


params_grid = {
    "loss" : ["exponential"],
    "n_estimators": [60,70,90], "max_depth": 10
              }
gboost = GradientBoostingClassifier()
gs = GridSearchCV(gboost, param_grid = params_grid, cv = 2 )
gs.fit(xtrain, ytrain)
report(gs.cv_results_)
gboost = gs.best_estimator_

In [0]:
gboost = GradientBoostingClassifier(loss='exponential', n_estimators=60, max_depth=6)

params_grid = {
    "loss" : ["deviance", "exponential"],
    "n_estimators": [10, 50,100, 150, 200],}

gs = GridSearchCV(gboost, param_grid = params_grid, cv = 5 )

gboost.fit(xtrain, ytrain)

gboost = gs.best_estimator_

In [0]:
gboost.fit( xtrain, ytrain)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=60,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [0]:
pred = gboost.predict(xtest)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.9280405405405405
-The recall Score:  0.8956343792633015
-The precision score:  0.956300072833212
-The F1_score:  0.9249735822472701
-The confusion matrix:
[[1434   60]
 [ 153 1313]]


```
PREVIOUS -The Accuracy of the the cllasifier trained only on youtube data: 0.9108108108108108
-The recall Score:  0.8997938144329897
-The precision score:  0.9214527027027027
-The F1_score:  0.9104944711036929
-The confusion matrix:
[[2199  186]
 [ 243 2182]]
 ```

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.9280405405405405
-The recall Score:  0.8956343792633015
-The precision score:  0.956300072833212
-The F1_score:  0.9249735822472701
-The confusion matrix:
[[1434   60]
 [ 153 1313]]


In [0]:
false_positive = []
false_positive_pred = []

false_negative = []
false_negative_pred = []

for i in range(len(pred)):
    ### False Positive check
    if ((pred[i]==1) & (ytest[i] ==0)):
        false_positive.append(i)
        false_positive_pred.append(pred[i])
    ### False Negative check
    elif ((pred[i] == 0) &(ytest[i]==1)):
        false_negative.append(i)
        false_negative_pred.append(pred[i])


In [0]:
print(len(false_positive))
print(len(false_negative))

60
153


False Positves

In [0]:
false_image_lst1 = list(np.array(paths_dict["test_paths1"])[false_positive])
false_image_lst2 = list(np.array(paths_dict["test_paths2"])[false_positive])

NameError: ignored

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt 

def showfalse(idx, i):
    
    try:
        imgs = [ plt.imread("data/images/" + false_image_lst1[idx]), 
               plt.imread("data/images/" + false_image_lst2[idx])]
        pathls = ["#: {} data/images/".format(i) + false_image_lst1[idx], 
                  "#: {} data/images/".format(i) + false_image_lst2[idx]]
        for i in range(len(imgs)):
            bb = detect_faces(imgs[i], m)
            ax = show_img(imgs[i])
            
            for b in bb:
                write_txt(ax, (b[0], b[1]),pathls[i], 14)
                draw_bb(ax, b)
            write_txt(ax, (0, 0),"{}".format(i), 20)
        #print(50*"#")
    except FileNotFoundError:
        print(i)

In [0]:
dump(gboost, "demo/PipelineParts/GboostModel.joblib")
dump(scaler, "demo/PipelineParts/scaler.joblib")
dump(f_selector, "demo/PipelineParts/feature_selector.joblib")

['demo/PipelineParts/feature_selector.joblib']

In [0]:
gboost

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=60,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [0]:
gboost = load("demo/PipelineParts/GboostModel.joblib")

In [0]:
pred = gboost.predict(xtest)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.9280405405405405
-The recall Score:  0.8956343792633015
-The precision score:  0.956300072833212
-The F1_score:  0.9249735822472701
-The confusion matrix:
[[1434   60]
 [ 153 1313]]
